In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# მუნიციპალიტეტების და მათში შემავალი ერთეულების ბაზა რეესტრიდან
munic_xl = pd.ExcelFile('export.xls')
list_active = [name for name in munic_xl.sheet_names if munic_xl.parse(name).legalStatusName[0] == 'აქტიური']

# print(munic_xl.sheet_names)

In [3]:
# გავწმინდოთ მუნიციპალიტეტების და მათში შემავალი ერთეულების ბაზა
munic_list = [{('ქალაქ თბილისის მუნიციპალიტეტი', 'წყნეთი', 'დაბა'),
              ('ქალაქ თბილისის მუნიციპალიტეტი', 'დიდი ლილო', 'დაბა'),
              ('ქალაქ თბილისის მუნიციპალიტეტი', 'კოჯორი', 'დაბა')}]
for name in list_active:
    df = munic_xl.parse(name, skiprows=3, usecols=3)
    name_cleaned = re.sub(string=name, pattern=' \w*$', repl=' მუნიციპალიტეტი')
    name_cleaned = re.sub(string=name_cleaned, pattern='^\d{1,2} ', repl='')
    munic_list.append(set([(name_cleaned, df['დასახლებათა დასახელება'][i],df['კატეგორია'][i]) 
                        for i in range(len(df)) 
                        if df['კატეგორია'][i] in ['დაბა', 'ქალაქი', 'სოფელი']]))

munic_list = list(set.union(*munic_list))

munic_df = pd.DataFrame(munic_list, columns=['მუნიციპალიტეტი', 'დასახელება', 'დასახლების_ტიპი'])

munic_df['მუნიციპალიტეტი_ტოპონიმი'] = munic_df['მუნიციპალიტეტი'].str.replace('ქალაქ ', '').str.replace('ს მუნიციპალიტეტი', '')
ending_on_a = ['თიღვი', 'წალენჯიხი', 'მესტიი', 'ახმეტი', 'მცხეთი', 'ქედი', 'თერჯოლი', 'სამტრედიი', 
               'ასპინძი', 'ჭიათური', 'წალკი', 'აბაში', 'ნინოწმინდი', 'ქურთი']
munic_df['მუნიციპალიტეტი_ტოპონიმი'] = np.where(munic_df['მუნიციპალიტეტი_ტოპონიმი'].isin(['თიღვი', 'წალენჯიხი', 'მესტიი', 'ახმეტი', 'მცხეთი', 'ქედი', 'თერჯოლი', 'სამტრედიი', 
               'ასპინძი', 'ჭიათური', 'წალკი', 'აბაში', 'ნინოწმინდი', 'ქურთი']), 
                                                munic_df['მუნიციპალიტეტი_ტოპონიმი'].str.replace('ი$', 'ა'), munic_df['მუნიციპალიტეტი_ტოპონიმი'])

munic_df['მუნიციპალიტეტი_ტოპონიმი'] = munic_df['მუნიციპალიტეტი_ტოპონიმი'].replace({'ახალციხი':'ახალციხე',
                                                                                           'საჩხერი':'საჩხერე',
                                                                                      'ყვარლი':'ყვარელი',
                                                                                       'გარდაბნი':'გარდაბანი', r'აჟარის \(ზემო აფხაზეთის\) მუნიციპალიტეტი':'აჟარა'}, regex=True)
munic_df['დასახელება'] = munic_df['დასახელება'].replace({'ქვ. |ქვ/|ქ/':'ქვემო ', 'ზ/|ზ. ':'ზემო ', 
                                                             '\n| I|II |I |მე-2 | \d|\.|\[1\]|\xa0':'', ' - ':'-', ' *\(.*\)$':''}, regex=True)

from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=['სოფელი', 'დაბა', 'ქალაქი'], ordered=True)
munic_df['დასახლების_ტიპი'] = munic_df['დასახლების_ტიპი'].astype(cat_type)

munic_df = munic_df.sort_values(by='დასახლების_ტიპი', axis=0, ascending=False).drop_duplicates(subset=['მუნიციპალიტეტი', 'დასახელება'], keep='first')

qalaqebi = list(munic_df['მუნიციპალიტეტი'][munic_df['მუნიციპალიტეტი'].str.contains('ქალაქ ')].replace({'ქალაქ ':''}, regex=True).unique())

munic_df = munic_df[~((munic_df['დასახელება'] == munic_df['მუნიციპალიტეტი_ტოპონიმი']) & munic_df['მუნიციპალიტეტი'].isin(qalaqebi))]

# ყაზბეგს ახლა სტეფანწმინდა ქვია მაგრამ ვაკანსიებში მაინც წერენ ყაზბეგს და ამიტომ დავამატოთ
kazbegi = {'მუნიციპალიტეტი':'ყაზბეგის მუნიციპალიტეტი', 'დასახელება':'ყაზბეგი', 'დასახლების_ტიპი':'დაბა', 'მუნიციპალიტეტი_ტოპონიმი':'ყაზბეგი'}
munic_df = munic_df.append(kazbegi, ignore_index=True)
zeindari = {'მუნიციპალიტეტი':'ვანის მუნიციპალიტეტი', 'დასახელება':'ზეინდარი', 'დასახლების_ტიპი':'სოფელი', 'მუნიციპალიტეტი_ტოპონიმი':'ვანი'}
munic_df = munic_df.append(zeindari, ignore_index=True)
shovi = {'მუნიციპალიტეტი':'ონის მუნიციპალიტეტი', 'დასახელება':'შოვი', 'დასახლების_ტიპი':'სოფელი', 'მუნიციპალიტეტი_ტოპონიმი':'ონი'}
munic_df = munic_df.append(shovi, ignore_index=True)

In [4]:
# მხარეების და მათში შემავალი მუნიციპალიტეტების ბაზა ვიკიპედიადან 
mkhare = pd.read_excel('population_munic.xlsx', sheet='1', names=['მუნიციპალიტეტი', 'მუნიციპალიტეტის_მოსახლეობა', 'მხარე'])
mkhare.set_index('მუნიციპალიტეტი', inplace=True)

set(mkhare.index) == set(munic_df['მუნიციპალიტეტი'])

True

In [5]:
# მთავარი საძიებო ბაზა საქართველოს ტოპონიმების (შევაერთოთ წინა ორი ბაზა)
ge_regions = munic_df.join(mkhare, on='მუნიციპალიტეტი', how='left')
ge_regions['ქვეყანა'] = 'საქართველო'

# მხოლოდ მუნიციპალიტეტეიბი განცხადების ტექსტში საძიებოდ
ge_regions_munic = ge_regions[ge_regions['მუნიციპალიტეტი_ტოპონიმი'] == ge_regions['დასახელება']]

# მდებარეობის ველშის გასაწმენდი dict
munic_dict = {ge_regions_munic['მუნიციპალიტეტი'][i].replace('ქალაქ ',''):ge_regions_munic['მუნიციპალიტეტი_ტოპონიმი'][i] for i in ge_regions_munic.index}

len(ge_regions_munic)

69

In [6]:
# მსოფლიოს ყვეყნების, დედაქალაქების და უდიდესი ქალაქების საძიებო ბაზა ვიკიპედიადან
# დასახელების ველში ქვეყნების სახელებიც შედის
intl_cities = pd.read_excel('wiki_capitals_largecities.xlsx', names=['დასახელება', 'ქვეყანა', 'დასახლების_ტიპი'])
uckhoeti = {'დასახელება':'უცხოეთი', 'ქვეყანა':'უცხოეთი', 'დასახლების_ტიპი':'ქვეყანა'}
intl_cities = intl_cities.append(uckhoeti, ignore_index=True)
intl_cities['ქვეყანა'] = intl_cities['ქვეყანა'].replace({'\n| I|II |I | \d|\.|\[1\]|\xa0':'', ' - ':'-', ' *\(.*\)$':''}, regex=True)
intl_cities['დასახელება'] = intl_cities['დასახელება'].replace({'\n| I|II |I | \d|\.|\[1\]|\xa0':'', ' - ':'-', ' *\(.*\)$':''}, regex=True)

intl_cities['მუნიციპალიტეტი'] = intl_cities['ქვეყანა']
intl_cities['მხარე'] = intl_cities['ქვეყანა']
intl_cities['მუნიციპალიტეტის_მოსახლეობა'] = np.nan
intl_cities['მუნიციპალიტეტი_ტოპონიმი'] = intl_cities['ქვეყანა']
intl_cities = intl_cities[ge_regions.columns]

intl_cities = intl_cities.drop_duplicates(subset=['დასახელება'], keep='first')

intl_cities.shape

(434, 7)

In [7]:
# ჩვენი ბაზა
data = pd.read_excel('filename.xlsx')

In [8]:
# გავწმინდოთ მდებარეობის ველი (ეს ყოველი ახალი ბაზისთვის იქნება შესავსები/განსაახლებელი): 
# გვინდა გვქონდეს მხოლოდ ტოპონიმები (ge_regions['დასახელება']-ში და intl_cities['დასახელება']-ში მოსაძებნად) და მხარეების სახელები ქვემოთ განმარტებული mkhareebi-დან 
data['მდებარეობა_cleaned'] = data['მდებარეობა'].replace({'მცხეთის, რ-ნი':'მცხეთის მუნიციპალიტეტი', 'რ-ნი':'მუნიციპალიტეტი', 'რაიონი':'მუნიციპალიტეტი',
                                                         'მუნიციპალიტეტის* სოფ(\.|ელი|ელ)':'მუნიციპალიტეტი,',
                                                        ', \w* ადმინისტრაციული ერთეული':'',
                                                        '/':', ',
                                                        'თბილისი - ქუთაისი':'თბილისი, ქუთაისი',
                                                        'საქართველო-ბულგარეთი':'საქართველო, ბულგარეთი',
                                                        'დას\.':'დასავლეთ',
                                                        'ს. |\.$|სოფელი |სოფ\.\s*|ქ\.\s*':'',
                                                        'ქვ\.\s*':'ქვემო ',
                                                        'ლეტეხი\.ჩიხარეში':'ლენტეხი, ჩიხარეში',
                                                        'აჭარა და სამცხე-ჯავახეთის რეგიონი':'აჭარის ავტონომიური რესპუბლიკა, სამცხე-ჯავახეთი',
                                                        'აჭარა':'აჭარის ავტონომიური რესპუბლიკა',
                                                        'გერმანიის ფედერაციული რესპუბლიკა':'გერმანია',
                                                        'გორი ქარელი':'გორი, ქარელი',
                                                        'დაბა ':'',
                                                        'თბილიან ქუთაისი':'თბილისი, ქუთაისი',
                                                        'დიდი ბრიტანეთიდა ჩრდილოეთ ირლანდიის გაერთიანებული სამეფო|დიდი ბრიტანეთისა და ჩრდილოეთ ირლანდიის გაერთიანებული სამეფო':'გაერთიანებული სამეფო',
                                                        ', დავით აღმაშენებლის მოედანი.*':'',
                                                        'ვარკეთილი|სააკაძის მოედანი|საბურთალო':'თბილისი',
                                                        'თელავში':'თელავი',
                                                        'თურქეთის რესპუბლიკა':'თურქეთი',
                                                        'ნატახატარი':'ნატახტარი',
                                                        'ოზურგუთი':'ოზურგეთი',
                                                        'რუსთავში':'რუსთავი',
                                                        'ლენტეხი.ჩიხარეში':'ლენტეხი, ჩიხარეში',
                                                        'სვანეთი':'სამეგრელო-ზემო სვანეთი',
                                                        'რაჭა':'რაჭა-ლეჩხუმი და ქვემო სვანეთი',
                                                        'სამეგრელო, ზემო სვანეთი|სამეგრელო|ზემო სვანეთი':'სამეგრელო-ზემო სვანეთი',
                                                        'ქვემო სამეგრელო-ზემო სვანეთი|რაჭა-ლეჩხუმი და ქვემო სვანეთი-ლეჩხუმი|რაჭა-ლეჩხუმი და ქვემო სამეგრელო-ზემო სვანეთი|რაჭა-ლეჩხუმი, ქვ. სვანეთი|რაჭა-ლეჩხუმი, ქვემო სვანეთი':'რაჭა-ლეჩხუმი და ქვემო სვანეთი',
                                                        'სამეგრელო-ზემო სვანეთი-სამეგრელო-ზემო სვანეთი|სამეგრელო-ზემო სვანეთი-ზემო სამეგრელო-ზემო სვანეთი|ზემო სამეგრელო-ზემო სვანეთი|სამეგრელო-ზემო სვანეთი-ზემო სვანეთი|სამეგრელო-სამეგრელო-ზემო სვანეთი|სამეგრელო-ზემო სვანეთი-სამეგრელო-ზემო სვანეთი':'სამეგრელო-ზემო სვანეთი',
                                                        'სამეგრელო-ზემო სვანეთი-სამეგრელო-ზემო სვანეთი':'სამეგრელო-ზემო სვანეთი',
                                                        'რაჭა-ლეჩხუმი და ქვემო სვანეთი-სამეგრელო-ზემო სვანეთი':'რაჭა-ლეჩხუმი და ქვემო სვანეთი',
                                                        'სსიპ - ილია წინამძღვრიშვილის სახელობის საზოგადოებრივი კოლეჯი':'თბილისი',
                                                        'ფასანაური,თიანეთი':'ფასანაური, თიანეთი',
                                                        ', ფრიდონ ხალვაშის გამზირი.*':'',
                                                        'ქვემო იმერეთი|ზემო იმერეთი':'იმერეთი',
                                                        'ჩინეთის სახალხო რესპუბლიკა':'ჩინეთი',
                                                        'ცენტრალური ფილიალი':'თბილისი',
                                                        'წეროვანის დევნილთა დასახლება':'წეროვანი',
                                                        'წითელი ხიდი':'მარნეულის მუნიციპალიტეტი',
                                                        'მტკვარი':'გარდაბნის მუნიციპალიტეტი',
                                                        'ნახატხარი':'ნატახტარი',
                                                        'გონიო|კვარიათი|მწვანე კონცხი':'ბათუმი',
                                                        'სამცხე ჯავახეთი':'სამცხე-ჯავახეთი',
                                                        'ქვემო ბოდბე':'ბოდბე',
                                                        'თუშეთი':'ომალო', # მივაკუთვნე კუთხის ცენტრს
                                                        'ზეთა':'ჯუთა', # ტურისტული ბანაკი ჯუთაში
                                                        'თელეთი':'ზემო თელეთი', # ძველი სახელი
                                                        'ჭუბერი':'ქვემო მარღი', # თემის დასახელება, მივაკუთვნე თემის ცენტრს
                                                        'ნეკრესი':'შილდა', # ძველი ქალაქი, შილდის მახლობლად
                                                        'ფონიჭალა|ლილო|300 არაგველი|ისანი|მარჯანიშვილის მოედანი|მარჯანიშვილი|ჩიტაია|დირსი|დიდი დიღომი|დიღომი|ვაკე':'თბილისი',
                                                        'ომანის სასულთნო':'ომანი',
                                                        'რიადი':'ერ-რიადი',
                                                        'ქუვეიტი':'ქუვეითი',
                                                        'ყატარი':'კატარი',
                                                        'ჰოლანდია':'ნიდერლანდები',
                                                        ',\s*':', ',
                                                        'აჩინებული':'თელავი' #საწარმო თელავში
                                                        }, regex=True)



data['მდებარეობა_cleaned'] = data['მდებარეობა_cleaned'].replace(munic_dict, regex=True)

In [10]:
#data[data['მდებარეობა_cleaned'].str.contains('თუშეთი')]
#ge_regions['დასახელება'][ge_regions['დასახელება'].str.contains('თუშეთი')]


In [9]:
# ჩვენს ბაზაში მოხსენიებული ყველა ტოპონიმის სია
toponyms = set([item for sublist in data['მდებარეობა_cleaned'].str.split(', ') for item in sublist])
print(toponyms)


{'საჩხერე', 'ენისელი', 'შიდა ქართლი', 'რაჭა-ლეჩხუმი და ქვემო სვანეთი', 'დოჰა', 'წნორი', 'ურეკი', 'ამბროლაური', 'ფოთი', 'ამერიკის შეერთებული შტატები', 'ორსანტია', 'ზეინდარი', 'შაქრიანი', 'ნორიო', 'ჩოხატაური', 'პრაღა', 'დმანისი', 'ასურეთი', 'რუისპირი', 'საქართველო', 'ქუთაისი', 'კაუნასი', 'ჰებეი', 'შუახევი', 'საგურამო', 'ჯიღაურა', 'ხულო', 'სიღნაღი', 'უკრაინა', 'მუხრანი', 'სიჩუანი', 'ადიგენი', 'მცხეთა-მთიანეთი', 'ცხვარიჭამია', 'ნუნისი', 'სურამი', 'თბილისი', 'ჰააგა', 'ახალციხე', 'ფოკა', 'ბაკურიანი', 'ძეგვი', 'ბანოჯა', 'გრემი', 'შაუმიანი', 'ტყიბული', 'ნავაზი', 'მაღრაანი', 'ტბისი', 'მანავი', 'ურაველი', 'ისრაელი', 'ბორჯომი', 'წალკა', 'შანხაი', 'სართიჭალა', 'დუშეთი', 'ახალქალაქი', 'ბოდბე', 'სარფი', 'ევროპა', 'ბულაჩაური', 'რუსეთი', 'აზერბაიჯანი', 'ლაფანყური', 'სამტრედია', 'გუდაური', 'სუფსა', 'ბათუმი', ' ', 'ბოლნისი', 'ყაზბეგი', 'აჭარისაღმართი', 'ვარდისუბანი', 'აღაიანი', 'ჯუთა', 'ყულევი', 'ლალისყური', 'უნგრეთი', 'გარდაბანი', 'ზესტაფონი', 'ჩეხეთი', 'ბაისუბანი', 'ვარშავა', 'ახმეტა', 'ბულგარეთი', 'კ

In [10]:
# მხარეების სია საძიებოდ
mkhareebi = {'საქართველო', 'აღმოსავლეთ საქართველო', 'დასავლეთ საქართველო'}.union(set(ge_regions['მხარე']))
mkhareebi.remove('თბილისი')
mkhareebi

{'აფხაზეთის ავტონომიური რესპუბლიკა',
 'აღმოსავლეთ საქართველო',
 'აჭარის ავტონომიური რესპუბლიკა',
 'გურია',
 'დასავლეთ საქართველო',
 'იმერეთი',
 'კახეთი',
 'მცხეთა-მთიანეთი',
 'რაჭა-ლეჩხუმი და ქვემო სვანეთი',
 'სამეგრელო-ზემო სვანეთი',
 'სამცხე-ჯავახეთი',
 'საქართველო',
 'ქვემო ქართლი',
 'შიდა ქართლი'}

In [11]:
ge_missing = ['ქედი']
not_found = toponyms - set(intl_cities['დასახელება']).union(ge_regions['დასახელება']).union(mkhareebi).union(ge_missing)
print(not_found)

{'სხვა', 'რეგიონები', ' '}


In [12]:
# ჩვენი ბაზის რიგების დასამუშავებელი ფუნქციები

def find_toponyms(toponym, text):
    # ტექსტში მოვძებნოთ ტოპონიმი ბრუნებების გათვალისწინებით 
    if toponym.endswith('ი'):
        result = bool(re.search(pattern='\W'+toponym[:-1]+'(ს\W|სა\W|ში\W|ი\W)', string=text))
    else:
        result = bool(re.search(pattern='\W'+toponym+'(ს\W|სა\W|ში\W|\W)', string=text))
    
    return result


def process_regions(row_num):
    # ჩვენი ბაზის row_num-ის რიგში მითითებული მდებარეობების სია
    regions = data.iloc[row_num]['მდებარეობა_cleaned'].split(', ')
    # ჩვენი ბაზის row_num-ის რიგის განცხადების ტექსტი
    text = data.iloc[row_num]['ვაკანსიის_დეტალები']
    
    # regions-ის მდებარეობებს ვადარებთ ტოპონიმების ბაზებს რომ მივიღოთ შემდეგი სიები: 
    # საქართველოს ტოპონიმების (ინდექსების) სია
    new_regions = []
    # საქართველოს მხარეების სია როცა სხვა ტოპონიმი არ მოიძებნა
    new_mkhare = []
    # უცხოური ტოპონიმების (ინდექსების) სია რაც ქართულ ბაზაში არ მოიძებნა
    new_intl = []
    # ტოპონიმების სია რომლებიც არც ერთ ბაზაში არ მოიძებნა
    garbage = []
    
    
    
    if regions == [' ']:
        # განცხადების ტექსტში მოვძებნოთ მუნიციპალიტეტის ტოპონიმები ბრუნებების გათვალისწინებით 
        toponyms_text = ge_regions_munic['დასახელება'][ge_regions_munic['დასახელება'].apply(lambda x: find_toponyms(x, text))].index.tolist()
        if len(toponyms_text) == 0:
            new_regions = [ge_regions[ge_regions['დასახელება'] == 'თბილისი'].index.values[0]]  # დავამატოთ 'თბილისი' თუ მდებარეობის ველი ცარიელია და ტექსტში არაფერი მოიძებნა
        else:
            new_regions = toponyms_text
    else:
        if len(regions) == 2:
            # მდებარეობის ველში 2 ტოპონიმის შემთხვევაში, შევამოწმოთ გვაქვს თუ არა 'ტოპონიმი, მუნიციპალიტეტი' წყვილი
            j=-1
            for i in range(len(ge_regions)):
                if set(regions).issubset(set(ge_regions.loc[i,:])):
                    j=i
                    break
                else:
                    pass
            
            if j == -1:
                # თუ არ გვაქვს 'ტოპონიმი, მუნიციპალიტეტი' წყვილი, სათითაოდ შევამოწმოთ მდებარეობის ველში მითითებული ყველა ტოპონიმი
                for region in regions:
                    if region in ['რეგიონები', 'სხვა']:
                        # განცხადების ტექსტში მოვძებნოთ მუნიციპალიტეტის ტოპონიმები ბრუნებების გათვალისწინებით 
                        toponyms_text = ge_regions_munic['დასახელება'][ge_regions_munic['დასახელება'].apply(lambda x: find_toponyms(x, text))].index.tolist()
                        new_regions = new_regions + toponyms_text
                    elif region in mkhareebi:
                        # განცხადების ტექსტში მოვძებნოთ მუნიციპალიტეტის ტოპონიმები ბრუნებების გათვალისწინებით 
                        toponyms_text = ge_regions_munic['დასახელება'][ge_regions_munic['დასახელება'].apply(lambda x: find_toponyms(x, text))].index.tolist()
                        if ((len(toponyms_text) == 0) or (toponyms_text == [ge_regions[ge_regions['დასახელება'] == 'თბილისი'].index.values[0]])):
                            # თუ ვერ ვიპოვეთ ვერაფერი ტექსტში, მაშინ მხარის სახელს ვინახავთ მხარეების სიაში
                            new_mkhare.append(region)
                        else:    
                            new_regions = new_regions + toponyms_text
                    else:
                        toponym_index = ge_regions[ge_regions['დასახელება'] == region].index.tolist()
                        if len(toponym_index) > 0:
                            new_regions = new_regions + [toponym_index[0]]
                        else:
                            intl_index = intl_cities[intl_cities['დასახელება'] == region].index.tolist()
                            if len(intl_index) > 0:
                                new_intl = [intl_index[0]]
                            else:
                                # თუ მდებარეობის ველში მითითებული ტოპონიმი არ მოიძებნა ბაზებში, ვინახავთ ცალკე
                                garbage.append(region)
                
            else:
                # თუ გვაქვს 'ტოპონიმი, მუნიციპალიტეტი' წყვილი, ვიყენებთ ზემოთ ნაპოვნ ინდექსს
                new_regions = [j]
            
        else: 
            # სათითაოდ შევამოწმოთ მდებარეობის ველში მითითებული ყველა ტოპონიმი (2-ზე მეტი ტოპონიმის შემთხვევაში)    
            for region in regions:
                if region in ['რეგიონები', 'სხვა']:
                    # განცხადების ტექსტში მოვძებნოთ მუნიციპალიტეტის ტოპონიმები ბრუნებების გათვალისწინებით 
                    toponyms_text = ge_regions_munic['დასახელება'][ge_regions_munic['დასახელება'].apply(lambda x: find_toponyms(x, text))].index.tolist()
                    new_regions = new_regions + toponyms_text
                elif region in mkhareebi:
                    # განცხადების ტექსტში მოვძებნოთ მუნიციპალიტეტის ტოპონიმები ბრუნებების გათვალისწინებით 
                    toponyms_text = ge_regions_munic['დასახელება'][ge_regions_munic['დასახელება'].apply(lambda x: find_toponyms(x, text))].index.tolist()
                    if ((len(toponyms_text) == 0) or (toponyms_text == [ge_regions[ge_regions['დასახელება'] == 'თბილისი'].index.values[0]])):
                        # თუ ვერ ვიპოვეთ ვერაფერი ტექსტში, მაშინ მხარის სახელს ვინახავთ მხარეების სიაში
                        new_mkhare.append(region)
                    else:    
                        new_regions = new_regions + toponyms_text
                else:
                    toponym_index = ge_regions[ge_regions['დასახელება'] == region].index.tolist()
                    if len(toponym_index) > 0:
                        new_regions = new_regions + [toponym_index[0]]
                    else:
                        intl_index = intl_cities[intl_cities['დასახელება'] == region].index.tolist()
                        if len(intl_index) > 0:
                            new_intl = [intl_index[0]]
                        else:
                            # თუ მდებარეობის ველში მითითებული ტოპონიმი არ მოიძებნა ბაზებში, ვინახავთ ცალკე
                            garbage.append(region)
    
    # მოვაშოროთ შესაძლო განმეორებები
    new_regions = list(set(new_regions))
    new_mkhare = list(set(new_mkhare))
    new_intl = list(set(new_intl))
    garbage = list(set(garbage))
    
    # ავაწყოთ ოთხივე სიის ცხრილები 
    df_ge = pd.concat([data.iloc[[row_num]*len(new_regions),:].reset_index(drop=True), ge_regions.loc[new_regions,:].reset_index(drop=True)], axis=1)
    df_intl = pd.concat([data.iloc[[row_num]*len(new_intl),:].reset_index(drop=True), intl_cities.loc[new_intl,:].reset_index(drop=True)], axis=1)   
    
    mkhare_df = pd.DataFrame({'მუნიციპალიტეტი':[np.nan]*len(new_mkhare), 'დასახელება':[np.nan]*len(new_mkhare), 'დასახლების_ტიპი':[np.nan]*len(new_mkhare),
       'მუნიციპალიტეტი_ტოპონიმი':[np.nan]*len(new_mkhare), 'მუნიციპალიტეტის_მოსახლეობა':[np.nan]*len(new_mkhare), 'მხარე':new_mkhare,
       'ქვეყანა':['საქართველო']*len(new_mkhare)})
    garbage_df = pd.DataFrame({'მუნიციპალიტეტი':[np.nan]*min(len(garbage),1), 'დასახელება':['ბაზაში არ გვაქვს: ' + ', '.join(garbage)]*min(len(garbage),1), 'დასახლების_ტიპი':[np.nan]*min(len(garbage),1),
       'მუნიციპალიტეტი_ტოპონიმი':[np.nan]*min(len(garbage),1), 'მუნიციპალიტეტის_მოსახლეობა':[np.nan]*min(len(garbage),1), 'მხარე':[np.nan]*min(len(garbage),1),
       'ქვეყანა':[np.nan]*min(len(garbage),1)})
    
    df_mkhare = pd.concat([data.iloc[[row_num]*len(new_mkhare),:].reset_index(drop=True), mkhare_df.reset_index(drop=True)], axis=1)   
    df_garbage = pd.concat([data.iloc[[row_num]*min(len(garbage),1),:].reset_index(drop=True), garbage_df.reset_index(drop=True)], axis=1)   

    # ახალი ცხრილი რამაც უნდა ჩაანაცვლოს row_num-ში მოცემული რიგი
    df = pd.concat([df_ge, df_intl, df_mkhare, df_garbage], axis=0, ignore_index=True)
    
    
    # თუ ადგილების რაოდენობა მითითებულია ჩვენ ბაზაში, ვამრავლებთ df ცხრილს შესაბამისად 
    num_vac = data.iloc[row_num]['ადგილების_რაოდენობა']
    
    if (not np.isnan(num_vac)) and  (num_vac // len(df)) > 1:
        # თუ ადგილების რაოდენომა გაყოფილი მდებარეობების რაოდენობაზე წილადია მაშინ ვამრგვალებთ ქვემოთ და ისე ვამრავლებთ df ცხრილს
        df_list = [df]*int(num_vac // len(df))
        df_list.append(df[:int(num_vac % len(df))])
        df = pd.concat(df_list, axis=0, ignore_index=True)
    else:
        pass
    
    
    return df
      

## ტესტირება

შევამოწმეთ ფუნქცია 200-200 განაცხადისთვის თითოეული წყაროდან და სწორად მუშაობს. 

In [13]:
# შევამოწმოთ როგორ მუშაობს: hr.gov.ge-ს 200 ვაკანსია
gov_test = pd.concat(list(map(process_regions, range(200))), axis=0, ignore_index=True)
gov_test[['source', 'ვაკანსია', 'დამსაქმებელი', 'მდებარეობა', 'ადგილების_რაოდენობა', 'მდებარეობა_cleaned',
                'მუნიციპალიტეტი', 'დასახელება', 'დასახლების_ტიპი', 'მუნიციპალიტეტი_ტოპონიმი', 
                'მუნიციპალიტეტის_მოსახლეობა', 'მხარე', 'ქვეყანა']]

,source,ვაკანსია,დამსაქმებელი,მდებარეობა,ადგილების_რაოდენობა,მდებარეობა_cleaned,მუნიციპალიტეტი,დასახელება,დასახლების_ტიპი,მუნიციპალიტეტი_ტოპონიმი,მუნიციპალიტეტის_მოსახლეობა,მხარე,ქვეყანა
0,gov,ინფრასტრუქტურის განვითარებისა და ლოჯისტიკის სა...,სსიპ - იუსტიციის სახლი,თბილისი,1.0,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
1,gov,სპეციალისტი გურჯაანის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,გურჯაანი,1.0,გურჯაანი,გურჯაანის მუნიციპალიტეტი,გურჯაანი,ქალაქი,გურჯაანი,53.104,კახეთი,საქართველო
2,gov,სივრცის კონსულტანტი მარნეულის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,მარნეული,1.0,მარნეული,მარნეულის მუნიციპალიტეტი,მარნეული,ქალაქი,მარნეული,106.454,ქვემო ქართლი,საქართველო
3,gov,სივრცის კონსულტანტი ზუგდიდის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,ზუგდიდი,1.0,ზუგდიდი,ქალაქ ზუგდიდის მუნიციპალიტეტი,ზუგდიდი,ქალაქი,ზუგდიდი,42.378,სამეგრელო-ზემო სვანეთი,საქართველო
4,gov,სივრცის კონსულტანტი ბათუმის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,ბათუმი,1.0,ბათუმი,ქალაქ ბათუმის მუნიციპალიტეტი,ბათუმი,ქალაქი,ბათუმი,163.440,აჭარის ავტონომიური რესპუბლიკა,საქართველო
5,gov,სივრცის კონსულტანტი ოზურგეთის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,ოზურგეთი,1.0,ოზურგეთი,ქალაქ ოზურგეთის მუნიციპალიტეტი,ოზურგეთი,ქალაქი,ოზურგეთი,14.645,გურია,საქართველო
6,gov,უფროსი სპეციალისტი (II კატეგორია) რუსთავის იუს...,სსიპ - იუსტიციის სახლი,რუსთავი,1.0,რუსთავი,ქალაქ რუსთავის მუნიციპალიტეტი,რუსთავი,ქალაქი,რუსთავი,127.839,ქვემო ქართლი,საქართველო
7,gov,სპეციალისტი თბილისის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,თბილისი,1.0,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
8,gov,სივრცის კონსულტანტი თბილისის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,თბილისი,4.0,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
9,gov,სივრცის კონსულტანტი თბილისის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,თბილისი,4.0,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო


In [14]:
# შევამოწმოთ როგორ მუშაობს: jobs.ge-ს 200 ვაკანსია
jobs_test = pd.concat(list(map(process_regions, range(35000,35200))), axis=0, ignore_index=True)
jobs_test[['source', 'ვაკანსია', 'დამსაქმებელი', 'მდებარეობა', 'ადგილების_რაოდენობა', 'მდებარეობა_cleaned',
                 'მუნიციპალიტეტი', 'დასახელება', 'დასახლების_ტიპი', 'მუნიციპალიტეტი_ტოპონიმი', 
                 'მუნიციპალიტეტის_მოსახლეობა', 'მხარე', 'ქვეყანა']]

,source,ვაკანსია,დამსაქმებელი,მდებარეობა,ადგილების_რაოდენობა,მდებარეობა_cleaned,მუნიციპალიტეტი,დასახელება,დასახლების_ტიპი,მუნიციპალიტეტი_ტოპონიმი,მუნიციპალიტეტის_მოსახლეობა,მხარე,ქვეყანა
0,jobs,ადმინისტრატორი,სასტუმრო ტიფლის მეტეხი,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
1,jobs,ძიძა,საბავშვო ბაღი,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
2,jobs,მიმტანი,ჭრელი აბანო,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
3,jobs,ადმინისტრატორი,ჭრელი აბანო,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
4,jobs,დამლაგებელი,ჭრელი აბანო,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
5,jobs,პრობლემური ქოლ-ცენტრის სპეციალისტი,ევროკრედიტი,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
6,jobs,ბაგა-ბაღის ადმინისტრატორი,საბა-22,ბათუმი,NaN,ბათუმი,ქალაქ ბათუმის მუნიციპალიტეტი,ბათუმი,ქალაქი,ბათუმი,163.440,აჭარის ავტონომიური რესპუბლიკა,საქართველო
7,jobs,"შეფ-მზარეული, მზარეული",კავკასიის ფრანგული სკოლა,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
8,jobs,კონსულტანტი,სანტექ სერვისი,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
9,jobs,მიმტანი,კაფეთა ქსელი პიატო,,NaN,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო


In [15]:
# შევამოწმოთ როგორ მუშაობს: hr.ge-ს 200 ვაკანსია
hr_test = pd.concat(list(map(process_regions, range(5000,5200))), axis=0, ignore_index=True)
hr_test[['source', 'ვაკანსია', 'დამსაქმებელი', 'მდებარეობა', 'ადგილების_რაოდენობა', 'მდებარეობა_cleaned',
               'მუნიციპალიტეტი', 'დასახელება', 'დასახლების_ტიპი', 'მუნიციპალიტეტი_ტოპონიმი', 
               'მუნიციპალიტეტის_მოსახლეობა', 'მხარე', 'ქვეყანა']]

,source,ვაკანსია,დამსაქმებელი,მდებარეობა,ადგილების_რაოდენობა,მდებარეობა_cleaned,მუნიციპალიტეტი,დასახელება,დასახლების_ტიპი,მუნიციპალიტეტი_ტოპონიმი,მუნიციპალიტეტის_მოსახლეობა,მხარე,ქვეყანა
0,hr,მოლარე-ოპერატორი,სიმკა პლასტი,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
1,hr,გაყიდვების აგენტი,სადაზღვევო კომპანია უნისონი,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
2,hr,კორპორატიული/B2B გაყიდვების ლიდერი,ეიესბისი,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
3,hr,Art Manager,Чайхона Базар,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
4,hr,Звукорежисер Караоке,Чайхона Базар,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
5,hr,Ведущий Караоке,Чайхона Базар,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
6,hr,დირექტორის თანაშემწე,საქართველოს ნავთობისა და გაზის კორპორაცია,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
7,hr,ოფისის მენეჯერი,ემბავუდ ჯორჯია,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
8,hr,დაცვის თანამშრომელი,უსაფრთხოების კომპანია ტიგონისი,თბილისი,NaN,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
9,hr,ჰარდ კოლექტორი ქუთაისში,პრობლემური აქტივების მართვის სააგენტო,ქუთაისი,NaN,ქუთაისი,ქალაქ ქუთაისის მუნიციპალიტეტი,ქუთაისი,ქალაქი,ქუთაისი,140.961,იმერეთი,საქართველო


In [16]:
writer = pd.ExcelWriter('tests_NOV06.xlsx')
gov_test.to_excel(writer,'gov')
hr_test.to_excel(writer,'hr')
jobs_test.to_excel(writer,'jobs')
writer.save()

## მთავარი ბაზის დამუშავება

ბაზა ძალიან დიდია, 55908 რიგით. აჯობებს 1000-1000 batch-ებით ვცადოთ. 

In [17]:
processed_batches = []
for i in range(int(len(data) // 1000) + min(1,int(len(data) % 1000))):
    processed_batch = pd.concat(list(map(process_regions, range(i*1000,min((i+1)*1000,len(data))))), axis=0, ignore_index=True)
    processed_batches.append(processed_batch)
    print('დამუშავდა {} ათასეული'.format(i+1))

დამუშავდა 1 ათასეული
დამუშავდა 2 ათასეული
დამუშავდა 3 ათასეული
დამუშავდა 4 ათასეული
დამუშავდა 5 ათასეული
დამუშავდა 6 ათასეული
დამუშავდა 7 ათასეული
დამუშავდა 8 ათასეული
დამუშავდა 9 ათასეული
დამუშავდა 10 ათასეული
დამუშავდა 11 ათასეული
დამუშავდა 12 ათასეული
დამუშავდა 13 ათასეული
დამუშავდა 14 ათასეული
დამუშავდა 15 ათასეული
დამუშავდა 16 ათასეული
დამუშავდა 17 ათასეული
დამუშავდა 18 ათასეული
დამუშავდა 19 ათასეული
დამუშავდა 20 ათასეული
დამუშავდა 21 ათასეული
დამუშავდა 22 ათასეული
დამუშავდა 23 ათასეული
დამუშავდა 24 ათასეული
დამუშავდა 25 ათასეული
დამუშავდა 26 ათასეული
დამუშავდა 27 ათასეული
დამუშავდა 28 ათასეული
დამუშავდა 29 ათასეული
დამუშავდა 30 ათასეული
დამუშავდა 31 ათასეული
დამუშავდა 32 ათასეული
დამუშავდა 33 ათასეული
დამუშავდა 34 ათასეული
დამუშავდა 35 ათასეული
დამუშავდა 36 ათასეული
დამუშავდა 37 ათასეული
დამუშავდა 38 ათასეული
დამუშავდა 39 ათასეული
დამუშავდა 40 ათასეული
დამუშავდა 41 ათასეული
დამუშავდა 42 ათასეული
დამუშავდა 43 ათასეული
დამუშავდა 44 ათასეული
დამუშავდა 45 ათასეული
დამუშავდა 46 ათასეუ

In [18]:
processed_data = pd.concat(processed_batches, axis=0, ignore_index=True)

In [20]:
print(len(processed_data))
processed_data[~processed_data['დასახელება'].isnull() & processed_data['დასახელება'].str.contains('ბაზაში არ გვაქვს')] 

68521


,source,ვაკანსია,დამსაქმებელი,გამოქვეყნდა,ბოლო_ვადა,მდებარეობა,ხელფასი,განათლება,გამოცდილება,ენები,...,Position,Employer,მდებარეობა_cleaned,მუნიციპალიტეტი,დასახელება,დასახლების_ტიპი,მუნიციპალიტეტი_ტოპონიმი,მუნიციპალიტეტის_მოსახლეობა,მხარე,ქვეყანა
52292,jobs,იურისტ-კონსულტანტი,საგანმანათლებლო-საკონსულტაციო ჯგუფი,2018-07-04,2018-07-14,"ქედი, ხულო",,,,,...,Lawyer-Consultant,Education and Consulting Group,"ქედი, ხულო",NaN,ბაზაში არ გვაქვს: ქედი,NaN,NaN,NaN,NaN,NaN


In [24]:
writer = pd.ExcelWriter('processed_regionNOV01.xlsx')
processed_data.to_excel(writer,'processed_regionNOV01')
writer.save()

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/167863/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/167860/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/167858/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/167859/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packag

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/165972/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/165971/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/165969/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/165968/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packag

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/164004/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/163997/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/163998/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/163996/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packag

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/13093' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/14270' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/12402' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/12970' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/l

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/19364' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/24690' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/19670' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'https://hr.ge/customer-details/18900' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/l

...

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/old-georgia-hotel/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/monitori-studio/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/sheni-marketi/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/nato-liaison-office/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per wo

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/bongiorno/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/gmc-pharm/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/ikamedi-poti/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/black-sea-group-company/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." %

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/lasare/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/noma-trans/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/socar/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/mp-group/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/an

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/link-logistic/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/fic-medical/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/falcon/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/pri/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/raz-imports/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/coffeshop-georgia/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/medical-city-hospital/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:840: UserWarning: Ignoring URL 'http://www.jobs.ge/eng/client/gts-electronics/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per wo

In [23]:
processed_data.to_csv('processed_regionNOV01.csv')

In [22]:
# დამუშავებული ბაზა
processed_read = pd.read_csv('processed_regionNOV01.csv')
processed_read.head()

,Unnamed: 0,source,ვაკანსია,დამსაქმებელი,გამოქვეყნდა,ბოლო_ვადა,მდებარეობა,ხელფასი,განათლება,გამოცდილება,...,Position,Employer,მდებარეობა_cleaned,მუნიციპალიტეტი,დასახელება,დასახლების_ტიპი,მუნიციპალიტეტი_ტოპონიმი,მუნიციპალიტეტის_მოსახლეობა,მხარე,ქვეყანა
0,0,gov,ინფრასტრუქტურის განვითარებისა და ლოჯისტიკის სა...,სსიპ - იუსტიციის სახლი,2018-10-26,2018-11-09,თბილისი,800 ლარი,სრული ზოგადი განათლება,3 წელი,...,,,თბილისი,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
1,1,gov,სპეციალისტი გურჯაანის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,2018-10-26,2018-11-09,გურჯაანი,550 ლარი,ბაკალავრი,6 თვე,...,,,გურჯაანი,გურჯაანის მუნიციპალიტეტი,გურჯაანი,ქალაქი,გურჯაანი,53.104,კახეთი,საქართველო
2,2,gov,სივრცის კონსულტანტი მარნეულის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,2018-10-26,2018-11-09,მარნეული,450 ლარი,სტუდენტი,,...,,,მარნეული,მარნეულის მუნიციპალიტეტი,მარნეული,ქალაქი,მარნეული,106.454,ქვემო ქართლი,საქართველო
3,3,gov,სივრცის კონსულტანტი ზუგდიდის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,2018-10-26,2018-11-09,ზუგდიდი,450 ლარი,სტუდენტი,,...,,,ზუგდიდი,ქალაქ ზუგდიდის მუნიციპალიტეტი,ზუგდიდი,ქალაქი,ზუგდიდი,42.378,სამეგრელო-ზემო სვანეთი,საქართველო
4,4,gov,სივრცის კონსულტანტი ბათუმის იუსტიციის სახლში,სსიპ - იუსტიციის სახლი,2018-10-26,2018-11-09,ბათუმი,450 ლარი,სტუდენტი,,...,,,ბათუმი,ქალაქ ბათუმის მუნიციპალიტეტი,ბათუმი,ქალაქი,ბათუმი,163.440,აჭარის ავტონომიური რესპუბლიკა,საქართველო


In [25]:
processed_read2 = pd.read_excel('processed_regionNOV01.xlsx')
processed_read2.tail(100)

,source,ვაკანსია,დამსაქმებელი,გამოქვეყნდა,ბოლო_ვადა,მდებარეობა,ხელფასი,განათლება,გამოცდილება,ენები,...,Position,Employer,მდებარეობა_cleaned,მუნიციპალიტეტი,დასახელება,დასახლების_ტიპი,მუნიციპალიტეტი_ტოპონიმი,მუნიციპალიტეტის_მოსახლეობა,მხარე,ქვეყანა
68421,jobs,გაყიდვების კონსულტანტი,ნავნე,2018-01-23,2018-02-23,რუსთავი,,,,,...,Sales Consultant,Navne,რუსთავი,ქალაქ რუსთავის მუნიციპალიტეტი,რუსთავი,ქალაქი,რუსთავი,127.839,ქვემო ქართლი,საქართველო
68422,jobs,იურისტის ასისტენტი,Divisor Group,2018-01-23,2018-02-23,,,,,,...,Assistant to Lawyer,Divisor Group,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
68423,jobs,მთავარი ბუღალტერი,კონკორდ თრეველი,2018-01-23,2018-02-23,,,,,,...,Chief Accountant,Concord Travel,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
68424,jobs,საკონდიტროს შეფი,სასტუმრო Rooms ყაზბეგი,2018-01-23,2018-02-24,ყაზბეგი,,,,,...,Pastry Chef,Rooms Hotel Kazbegi,ყაზბეგი,ყაზბეგის მუნიციპალიტეტი,ყაზბეგი,დაბა,ყაზბეგი,3.818,მცხეთა-მთიანეთი,საქართველო
68425,jobs,არა სიცოცხლის დაზღვევის ზარალების ანაზღაურების...,გლობალ ბენეფიტს ჯორჯია,2018-01-23,2018-02-22,,,,,,...,Head of Non-Life Insurance Claims Group,Global Benefits Georgia,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
68426,jobs,გაყიდვების სპეციალისტი,კლინიკა,2018-01-23,2018-02-23,,,,,,...,Sales Specialist,Clinic,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
68427,jobs,"მცხობელი, მცხობელის დამხმარე",ბეიკერ ლაინი,2018-01-23,2018-02-23,,,,,,...,Baker,Baker Line,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
68428,jobs,მთარგმნელი/რედაქტორი - ფრანგული,კომპანია,2018-01-23,2018-02-23,,,,,,...,Translator/Editor - French,Company,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო
68429,jobs,დისტრიბუტორი,აის ფუდ 2015,2018-01-23,2018-02-23,რუსთავი,,,,,...,Distributor,Ice Food 2015,რუსთავი,ქალაქ რუსთავის მუნიციპალიტეტი,რუსთავი,ქალაქი,რუსთავი,127.839,ქვემო ქართლი,საქართველო
68430,jobs,სხვადასხვა,ესთეტიკ სტუდიო,2018-01-23,2018-02-23,,,,,,...,Various,Esthetic Studio,,ქალაქ თბილისის მუნიციპალიტეტი,თბილისი,ქალაქი,თბილისი,1158.677,თბილისი,საქართველო


In [26]:
processed_read2.shape

(68521, 29)

In [39]:
#processed_read = processed_read.drop('Unnamed: 0', axis=1)
processed_read.shape

(68521, 29)

In [63]:
print(set(processed_read['ვაკანსია'][363]) - set(processed_read2['ვაკანსია'][363]))
print(processed_read2['ვაკანსია'][363])

{'\r'}
შიდა აუდიტის და ინსპექტირების სამსახურის I კატეგორიის უფროსი სპეციალისტი შიდა აუდიტის და

ინსპექტირების საკითხებში
